In [6]:
# Pandas shift() method is quite useful
from pandas import DataFrame
df = DataFrame()
df['t'] = [x for x in range(10)]
print(df)


   t
0  0
1  1
2  2
3  3
4  4
5  5
6  6
7  7
8  8
9  9


In [7]:
df['t-1'] = df['t'].shift(1)
print(df)

   t  t-1
0  0  NaN
1  1  0.0
2  2  1.0
3  3  2.0
4  4  3.0
5  5  4.0
6  6  5.0
7  7  6.0
8  8  7.0
9  9  8.0


In [9]:
df['t+1'] = df['t'].shift(-1)
print(df)

   t  t-1  t+1
0  0  NaN  1.0
1  1  0.0  2.0
2  2  1.0  3.0
3  3  2.0  4.0
4  4  3.0  5.0
5  5  4.0  6.0
6  6  5.0  7.0
7  7  6.0  8.0
8  8  7.0  9.0
9  9  8.0  NaN


In [20]:
from math import sqrt
from numpy import concatenate
from matplotlib import pyplot
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

Using TensorFlow backend.


In [61]:
# load dataset
dataset = read_csv('pollution.csv', header=0, index_col=0)
values = dataset.values
print (dataset)
print(values)


                     pollution  dew  temp   press wnd_dir  wnd_spd  snow  rain
date                                                                          
2010-01-02 00:00:00      129.0  -16  -4.0  1020.0      SE     1.79     0     0
2010-01-02 01:00:00      148.0  -15  -4.0  1020.0      SE     2.68     0     0
2010-01-02 02:00:00      159.0  -11  -5.0  1021.0      SE     3.57     0     0
2010-01-02 03:00:00      181.0   -7  -5.0  1022.0      SE     5.36     1     0
2010-01-02 04:00:00      138.0   -7  -5.0  1022.0      SE     6.25     2     0
2010-01-02 05:00:00      109.0   -7  -6.0  1022.0      SE     7.14     3     0
2010-01-02 06:00:00      105.0   -7  -6.0  1023.0      SE     8.93     4     0
2010-01-02 07:00:00      124.0   -7  -5.0  1024.0      SE    10.72     0     0
2010-01-02 08:00:00      120.0   -8  -6.0  1024.0      SE    12.51     0     0
2010-01-02 09:00:00      132.0   -7  -5.0  1025.0      SE    14.30     0     0
2010-01-02 10:00:00      140.0   -7  -5.0  1026.0   

In [40]:
# integer encode direction
encoder = LabelEncoder()
values[:, 4] = encoder.fit_transform(values[:, 4])
# ensure all data is float
values = values.astype('float32')
print(values)

[[129.   -16.    -4.   ...   1.79   0.     0.  ]
 [148.   -15.    -4.   ...   2.68   0.     0.  ]
 [159.   -11.    -5.   ...   3.57   0.     0.  ]
 ...
 [ 10.   -22.    -3.   ... 242.7    0.     0.  ]
 [  8.   -22.    -4.   ... 246.72   0.     0.  ]
 [ 12.   -21.    -3.   ... 249.85   0.     0.  ]]


In [47]:
# normalize features
scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(values)
print(scaled)
print(scaled.shape)


[[0.12977867 0.35294122 0.24590163 ... 0.00229001 0.         0.        ]
 [0.14889336 0.36764708 0.24590163 ... 0.00381099 0.         0.        ]
 [0.15995975 0.4264706  0.22950819 ... 0.00533197 0.         0.        ]
 ...
 [0.01006036 0.2647059  0.26229507 ... 0.41399646 0.         0.        ]
 [0.00804829 0.2647059  0.24590163 ... 0.4208665  0.         0.        ]
 [0.01207243 0.2794118  0.26229507 ... 0.42621556 0.         0.        ]]
(43800, 8)


In [66]:
print(scaled[0])

[0.12977867 0.35294122 0.24590163 0.5272732  0.6666667  0.00229001
 0.         0.        ]


In [52]:
data = scaled
n_in = 1
n_out = 1
n_vars = 1 if type(data) is list else data.shape[1]
print(n_vars)

8


In [94]:
df = DataFrame(data)
cols, names = list(), list()
# input sequence (t-n, ... t-1)
for i in range(n_in, 0, -1):
    cols.append(df.shift(i))
    names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]

    # forecast sequence (t, t+1, ... t+n)
for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
print(cols)
print(names)


[              0         1         2         3         4         5         6    7
0           NaN       NaN       NaN       NaN       NaN       NaN       NaN  NaN
1      0.129779  0.352941  0.245902  0.527273  0.666667  0.002290  0.000000  0.0
2      0.148893  0.367647  0.245902  0.527273  0.666667  0.003811  0.000000  0.0
3      0.159960  0.426471  0.229508  0.545454  0.666667  0.005332  0.000000  0.0
4      0.182093  0.485294  0.229508  0.563637  0.666667  0.008391  0.037037  0.0
5      0.138833  0.485294  0.229508  0.563637  0.666667  0.009912  0.074074  0.0
6      0.109658  0.485294  0.213115  0.563637  0.666667  0.011433  0.111111  0.0
7      0.105634  0.485294  0.213115  0.581818  0.666667  0.014492  0.148148  0.0
8      0.124748  0.485294  0.229508  0.600000  0.666667  0.017551  0.000000  0.0
9      0.120724  0.470588  0.213115  0.600000  0.666667  0.020610  0.000000  0.0
10     0.132797  0.485294  0.229508  0.618183  0.666667  0.023669  0.000000  0.0
11     0.140845  0.485294  

In [99]:
# put it all together
agg = concat(cols, axis=1)
agg.columns = names
agg

,var1(t-1),var2(t-1),var3(t-1),var4(t-1),var5(t-1),var6(t-1),var7(t-1),var8(t-1),var1(t),var2(t),var3(t),var4(t),var5(t),var6(t),var7(t),var8(t)
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.129779,0.352941,0.245902,0.527273,0.666667,0.002290,0.000000,0.0
1,0.129779,0.352941,0.245902,0.527273,0.666667,0.002290,0.000000,0.0,0.148893,0.367647,0.245902,0.527273,0.666667,0.003811,0.000000,0.0
2,0.148893,0.367647,0.245902,0.527273,0.666667,0.003811,0.000000,0.0,0.159960,0.426471,0.229508,0.545454,0.666667,0.005332,0.000000,0.0
3,0.159960,0.426471,0.229508,0.545454,0.666667,0.005332,0.000000,0.0,0.182093,0.485294,0.229508,0.563637,0.666667,0.008391,0.037037,0.0
4,0.182093,0.485294,0.229508,0.563637,0.666667,0.008391,0.037037,0.0,0.138833,0.485294,0.229508,0.563637,0.666667,0.009912,0.074074,0.0
5,0.138833,0.485294,0.229508,0.563637,0.666667,0.009912,0.074074,0.0,0.109658,0.485294,0.213115,0.563637,0.666667,0.011433,0.111111,0.0
6,0.109658,0.485294,0.213115,0.563637,0.666667,0.011433,0.111111,0.0,0.105634,0.485294,0.213115,0.581818,0.666667,0.014492,0.148148,0.0
7,0.105634,0.485294,0.213115,0.581818,0.666667,0.014492,0.148148,0.0,0.124748,0.485294,0.229508,0.600000,0.666667,0.017551,0.000000,0.0
8,0.124748,0.485294,0.229508,0.600000,0.666667,0.017551,0.000000,0.0,0.120724,0.470588,0.213115,0.600000,0.666667,0.020610,0.000000,0.0
9,0.120724,0.470588,0.213115,0.600000,0.666667,0.020610,0.000000,0.0,0.132797,0.485294,0.229508,0.618183,0.666667,0.023669,0.000000,0.0


In [118]:
# drop rows with NaN values
if True:
    agg.dropna(inplace=True)
agg

,var1(t-1),var2(t-1),var3(t-1),var4(t-1),var5(t-1),var6(t-1),var7(t-1),var8(t-1),var1(t)
1,0.129779,0.352941,0.245902,0.527273,0.666667,0.002290,0.000000,0.0,0.148893
2,0.148893,0.367647,0.245902,0.527273,0.666667,0.003811,0.000000,0.0,0.159960
3,0.159960,0.426471,0.229508,0.545454,0.666667,0.005332,0.000000,0.0,0.182093
4,0.182093,0.485294,0.229508,0.563637,0.666667,0.008391,0.037037,0.0,0.138833
5,0.138833,0.485294,0.229508,0.563637,0.666667,0.009912,0.074074,0.0,0.109658
6,0.109658,0.485294,0.213115,0.563637,0.666667,0.011433,0.111111,0.0,0.105634
7,0.105634,0.485294,0.213115,0.581818,0.666667,0.014492,0.148148,0.0,0.124748
8,0.124748,0.485294,0.229508,0.600000,0.666667,0.017551,0.000000,0.0,0.120724
9,0.120724,0.470588,0.213115,0.600000,0.666667,0.020610,0.000000,0.0,0.132797
10,0.132797,0.485294,0.229508,0.618183,0.666667,0.023669,0.000000,0.0,0.140845


In [108]:
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
	n_vars = 1 if type(data) is list else data.shape[1]
	df = DataFrame(data)
	cols, names = list(), list()
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i))
		names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(df.shift(-i))
		if i == 0:
			names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
		else:
			names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
	# put it all together
	agg = concat(cols, axis=1)
	agg.columns = names
	# drop rows with NaN values
	if dropnan:
		agg.dropna(inplace=True)
	return agg
    

In [123]:
reframed = agg
reframed

,var1(t-1),var2(t-1),var3(t-1),var4(t-1),var5(t-1),var6(t-1),var7(t-1),var8(t-1),var1(t)
1,0.129779,0.352941,0.245902,0.527273,0.666667,0.002290,0.000000,0.0,0.148893
2,0.148893,0.367647,0.245902,0.527273,0.666667,0.003811,0.000000,0.0,0.159960
3,0.159960,0.426471,0.229508,0.545454,0.666667,0.005332,0.000000,0.0,0.182093
4,0.182093,0.485294,0.229508,0.563637,0.666667,0.008391,0.037037,0.0,0.138833
5,0.138833,0.485294,0.229508,0.563637,0.666667,0.009912,0.074074,0.0,0.109658
6,0.109658,0.485294,0.213115,0.563637,0.666667,0.011433,0.111111,0.0,0.105634
7,0.105634,0.485294,0.213115,0.581818,0.666667,0.014492,0.148148,0.0,0.124748
8,0.124748,0.485294,0.229508,0.600000,0.666667,0.017551,0.000000,0.0,0.120724
9,0.120724,0.470588,0.213115,0.600000,0.666667,0.020610,0.000000,0.0,0.132797
10,0.132797,0.485294,0.229508,0.618183,0.666667,0.023669,0.000000,0.0,0.140845


In [124]:
# drop columns we don't want to predict

reframed = series_to_supervised(scaled, 1, 1)
reframed.drop(reframed.columns[[9,10,11,12,13,14,15]], axis=1, inplace=True)
reframed

,var1(t-1),var2(t-1),var3(t-1),var4(t-1),var5(t-1),var6(t-1),var7(t-1),var8(t-1),var1(t)
1,0.129779,0.352941,0.245902,0.527273,0.666667,0.002290,0.000000,0.0,0.148893
2,0.148893,0.367647,0.245902,0.527273,0.666667,0.003811,0.000000,0.0,0.159960
3,0.159960,0.426471,0.229508,0.545454,0.666667,0.005332,0.000000,0.0,0.182093
4,0.182093,0.485294,0.229508,0.563637,0.666667,0.008391,0.037037,0.0,0.138833
5,0.138833,0.485294,0.229508,0.563637,0.666667,0.009912,0.074074,0.0,0.109658
6,0.109658,0.485294,0.213115,0.563637,0.666667,0.011433,0.111111,0.0,0.105634
7,0.105634,0.485294,0.213115,0.581818,0.666667,0.014492,0.148148,0.0,0.124748
8,0.124748,0.485294,0.229508,0.600000,0.666667,0.017551,0.000000,0.0,0.120724
9,0.120724,0.470588,0.213115,0.600000,0.666667,0.020610,0.000000,0.0,0.132797
10,0.132797,0.485294,0.229508,0.618183,0.666667,0.023669,0.000000,0.0,0.140845
